# 実験２　葉のデータの機械学習  深層学習

## 学習対象データ

- １２８×１２８のカラー画像　
- カテゴリ数 ７
- カテゴリあたりのデータ数１６
- 総画像数１１２

## ネットワーク構成

1.  入力層 　　（縦１２８, 横128, 3チャンネル）  
2. たたみこみ層 　　３×３  × 64チャネル
3.  プーリング層　　max プーリング　　　２x２   ストライド２
4. ドロップアウト
5.  たたみこみ層 　　３×３  × 32チャネル
6.  プーリング層　　max プーリング　　　２x２   ストライド２
7. 全結合層  256
8. ドロップアウト
9. 全結合層　　32
10. 出力層（softmax 出力）

活性化関数としてはランプ関数（　ReLU関数）を用いる。


In [23]:
#  基礎実験
import numpy as np
from PIL import Image
import os, glob, random

imgsize = 128 # 画像のサイズ
ppics = 16  # カテゴリごとの画像数  

folders = glob.glob("pics/*")

categories = [os.path.basename(c) for c in folders]
classes = len(categories)

print("クラス数=",classes, categories)

クラス数= 7 ['hanamizuki', 'kinmokusei', 'yoshino', 'yamazakura', 'hamokuren', 'keyaki', 'hananoki']


In [24]:
# 画像の読み込み
def make_dataset():
    global X
    global y
    X, y = [],[]
    folders = glob.glob("pics/*")
    # print(folders)
    for i, folder in enumerate(folders):
        files = glob.glob(folder + "/*.jpg")
        for j, file in enumerate(files):
            img = Image.open(file)
            # img = img.convert("RGB")   # もし白黒画像が含まれているなら、変換が必要
            img = img.resize((imgsize,imgsize))
            data = np.asarray(img,dtype=np.float32)
            X.append(data)
        y = np.r_[y,np.array([i]*ppics).astype(np.int32)] 
    X = np.array(X)
    
# データセットの生成
make_dataset()
print("データ数は",len(X))
print("正解ラベル",y)

データ数は 112
正解ラベル [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  2.  2.  2.  2.
  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  3.  3.  3.  3.  3.  3.
  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  4.  4.  4.  4.  4.  4.  4.  4.
  4.  4.  4.  4.  4.  4.  4.  4.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.
  5.  5.  5.  5.  5.  5.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.
  6.  6.  6.  6.]


In [25]:
# データの標準化
#  経験的に、データは平均が０になっている方が効率よく学習できるので、標準化を施す。
# 元のデータは整数値であるが、標準化により実数になることに注意

from sklearn.preprocessing import StandardScaler
sc = [StandardScaler() for i in range(3)]
for i in range(3):
    sp = X[:,:,:,i].shape
    Xt = X[:,:,:,i].reshape(-1,1)
    sc[i].fit(Xt)
    X[:,:,:,i] = sc[i].transform(Xt).reshape(sp)   

# 学習

In [26]:
# from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from keras.models import Sequential
from keras.layers import Activation, Dropout, Dense,  Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPool2D
from keras.optimizers import Adam

from keras.utils import np_utils
import numpy as np

# 実験

学習データ１１２　（＝７カテゴリ×１６）をシャッフルしてカテゴリが偏らないように、 訓練データとテストデータに２等分し、
訓練データに対する正答率とテストデータに対する正答率を求める。

同じ検定を５回繰り返して正答率の平均値を求めてみる。

In [27]:
def main():
    result_train,result_test = [],[]
    sss = StratifiedShuffleSplit(n_splits=5, test_size=0.5, random_state=0)
    count = 0
    for train_index, test_index in sss.split(X,y):
        print(len(train_index),len(test_index))
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model = train(X_train, y_train)
        result_train.append(model_eval(model, X_train, y_train))
        result_test.append(model_eval(model, X_test, y_test))
        print('train loss=',result_train[count][0])
        print('train accuracy=',result_train[count][1])
        print('test loss=',result_test[count][0])
        print('test accuracy=',result_test[count][1])
        count = count+1
    return model,train_index,test_index,result_train,result_test
            
def train(X, y):
    model = Sequential()
    
    model.add(Conv2D(64,3,input_shape=(128,128,3)))
    model.add(Activation('relu'))
    model.add(MaxPool2D(pool_size=(2,2)))

    model.add(Conv2D(32,3))
    model.add(Activation('relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(classes,activation='softmax'))
    
    adam = Adam(lr=1e-4)
    
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=adam, metrics=['accuracy'])
    history = model.fit(X,y,epochs=20)
    return model

def model_eval(model, X, y):
    score = model.evaluate(X,y)
    return score

学習の実行

In [28]:
model,train_index,test_index,result_train,result_test = main()

56 56
Epoch 1/20
56/56 [==============================] - 6s 111ms/step - loss: 2.2677 - acc: 0.1607
Epoch 2/20
56/56 [==============================] - 4s 64ms/step - loss: 1.8328 - acc: 0.2143
Epoch 3/20
56/56 [==============================] - 3s 62ms/step - loss: 1.5311 - acc: 0.3929
Epoch 4/20
56/56 [==============================] - 4s 63ms/step - loss: 1.0415 - acc: 0.6786
Epoch 5/20
56/56 [==============================] - 4s 69ms/step - loss: 1.0795 - acc: 0.5893
Epoch 6/20
56/56 [==============================] - 4s 70ms/step - loss: 0.8359 - acc: 0.7321
Epoch 7/20
56/56 [==============================] - 4s 70ms/step - loss: 0.7088 - acc: 0.7679
Epoch 8/20
56/56 [==============================] - 4s 68ms/step - loss: 0.5666 - acc: 0.8750
Epoch 9/20
56/56 [==============================] - 5s 83ms/step - loss: 0.4844 - acc: 0.8393
Epoch 10/20
56/56 [==============================] - 4s 73ms/step - loss: 0.4061 - acc: 0.8393
Epoch 11/20
56/56 [==============================] -

56/56 [==============================] - 1s 16ms/step
train loss= 0.00533775230204
train accuracy= 1.0
test loss= 2.28623434475
test accuracy= 0.357142852885
56 56
Epoch 1/20
56/56 [==============================] - 7s 118ms/step - loss: 2.1647 - acc: 0.1964
Epoch 2/20
56/56 [==============================] - 4s 64ms/step - loss: 1.6218 - acc: 0.3036
Epoch 3/20
56/56 [==============================] - 4s 65ms/step - loss: 1.2512 - acc: 0.5357
Epoch 4/20
56/56 [==============================] - 4s 66ms/step - loss: 0.9495 - acc: 0.7679
Epoch 5/20
56/56 [==============================] - 4s 66ms/step - loss: 0.8494 - acc: 0.7500
Epoch 6/20
56/56 [==============================] - 3s 62ms/step - loss: 0.6712 - acc: 0.7679
Epoch 7/20
56/56 [==============================] - 4s 65ms/step - loss: 0.4729 - acc: 0.8929
Epoch 8/20
56/56 [==============================] - 4s 64ms/step - loss: 0.4241 - acc: 0.9107
Epoch 9/20
56/56 [==============================] - 3s 62ms/step - loss: 0.3109 - a

# 正認識率


In [20]:
trainaccav, testaccav = 0,0 
for i in range(5):
    print("{0}回目　　　　訓練データ正解率{1:5.２f}　 　テストデータ正解率{２:5.２f}".format(i,result_train[i][1], result_test[i][1]))
    trainaccav, testaccav = trainaccav+ result_train[i][1], testaccav+result_test[i][1]  

print("平均正認識率　　　　訓練データ　 {0:5.２f}　     テストデータ　 {1:5.２f}　　".format(trainaccav/5,testaccav/5))

0回目　　　　訓練データ正解率 1.00　 　テストデータ正解率 0.46
1回目　　　　訓練データ正解率 1.00　 　テストデータ正解率 0.38
2回目　　　　訓練データ正解率 1.00　 　テストデータ正解率 0.29
3回目　　　　訓練データ正解率 1.00　 　テストデータ正解率 0.36
4回目　　　　訓練データ正解率 1.00　 　テストデータ正解率 0.38
平均正認識率　　　　訓練データ　  1.00　     テストデータ　  0.37　　


# 詳細な認識結果
５回試みた学習のうち、最終回の学習結果による全データの認識結果

In [22]:
pred = model.predict_classes(X)
for i in range(7):
    for j in range(16):
        print("{}/{}".format(i,pred[16*i+j]), end="     ")
    print()

0/6     0/0     0/0     0/2     0/2     0/0     0/0     0/6     0/0     0/0     0/2     0/0     0/4     0/0     0/6     0/0     
1/5     1/5     1/1     1/1     1/6     1/5     1/1     1/1     1/1     1/1     1/1     1/5     1/1     1/1     1/1     1/1     
2/6     2/6     2/2     2/2     2/1     2/2     2/6     2/0     2/2     2/2     2/2     2/0     2/2     2/2     2/6     2/2     
3/3     3/4     3/3     3/5     3/3     3/3     3/1     3/2     3/3     3/3     3/3     3/3     3/3     3/3     3/3     3/1     
4/4     4/2     4/4     4/1     4/4     4/4     4/4     4/4     4/4     4/4     4/4     4/4     4/4     4/4     4/3     4/4     
5/5     5/5     5/5     5/5     5/6     5/5     5/5     5/5     5/5     5/5     5/1     5/5     5/1     5/5     5/6     5/4     
6/5     6/0     6/6     6/6     6/6     6/4     6/6     6/6     6/6     6/6     6/6     6/6     6/6     6/6     6/6     6/6     


# 誤認識の解析

一例として、最後のケースについて、 正解不正解の和人誤認識例を確認してみると、

||カテゴリ |正認識数 |誤認識数|誤認識されたカテゴリ|
|:--------------:|:-----------:|:----------:|:--------:|:---------------|
|0|ハナミズキ|9|7||
|1|キンモクセイ|１1|5||
|２|ソメイヨシノ|9|7|　|
|３|ヤマザクラ|１1|5||  
|４|ハモクレン|１3|3||
|５|ケヤキ|11|5||  
|６|ハナノキ|１3|3||